NOMBRE DE LETTRES AU PERE NOEL PAR PAYS

In [ ]:
import psycopg2
import os
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt


def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    #print(chemin)
    return chemin

# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        #print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        #print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn

def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")

In [ ]:
conn=connexion_DB("santa_data")
cursor = conn.cursor()

In [ ]:
#main
if(conn): 
    # compte du nombre de lettres où le pays est renseigné (=pays non null)
    rq_count='''
                SELECT count(RQ_L.id_l)
                FROM (SELECT DISTINCT id_l, id_p FROM T_enfants
                        WHERE id_p IS NOT NULL) AS RQ_L
                ;                       
            '''     
    cursor.execute(rq_count)
    nb_tot=cursor.fetchone() 
    
    # compte du nombre de lettres par Pays non Null
    rq_select='''
                SELECT count(id_l),pays
                FROM (SELECT DISTINCT id_l, id_p FROM T_enfants) AS RQ_L
                JOIN T_Pays ON RQ_L.id_p = T_pays.id_p
                GROUP BY pays
                ORDER BY count(id_l) DESC
                ;        
            '''     
cursor.execute(rq_select)
res=cursor.fetchall()

#mise en forme du résultat de la requete
print('\nNOMBRE DE LETTRES PAR PAYS NON NULL : ',nb_tot[0])
print('NOMBRE DE PAYS : ', cursor.rowcount)
print('\n                     PAYS','   NB_Lettres'.ljust(22),' % de ',nb_tot[0])
      
trait="-"*70
print(trait)
for row in res:
    print(str(row[1])[0:30].rjust(25),'       ',end='')
    print(str(row[0]).ljust(15), str(round(row[0]/nb_tot[0]*100,2)).rjust(8),'%')
    

In [ ]:
#Barplot horizontal :

#    on utilise barh, en remplaçant width par height et bottom par left.


nb_p_df=pd.read_sql_query ( rq_select , conn , index_col = None , coerce_float = True ,
                            params = None , parse_dates = None , chunksize = None )
r = range(len(nb_p_df))

fig=plt.figure(figsize=(25,35))
barWidth = 0.4

plt.barh(r, nb_p_df['count'], height = barWidth, color = ['yellow' for i in nb_p_df['count']],\
           edgecolor = ['blue' for i in nb_p_df['count']], linestyle = 'solid', hatch ='/',linewidth = 3)

plt.yticks(range(len(nb_p_df['count'])), nb_p_df['pays'])
#Dans un barplot, pour supprimer les ticks sur l'axe des x (tout en conservant les labels) : faire 
plt.gca().axes.xaxis.set_ticks_position('none')

plt.show()  